In [1]:
import pandas, csv, time
import tweepy
import webbrowser

consumer_key = 'urBfiZz4ztGR7pzLKCTpI3oML'
secret_key = 'OOPgRXDgpQhzneTrqcY63igbjkfdB2gqZ88ChGhj3kIo9VqKf2'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAJmHIwEAAAAAwR6Bwu%2BRRZmAQn9t%2Bao%2BlgFI5%2F4%3D7uQVJIHjuG1vPHZNfsdKJGpfGbWvLRCj50UFkA4TdF3iRRYtS1'
access_token = '1317128139261304832-43UyRfOPGcLWjS2M801BhqyANfXeNJ'
access_secret = '29FXOSp3GoAst8tIKgBZ52SOVTi7Ug7XHhDZMYRF5B5Kt'

callback_uri = 'oob'
#sign in to twitter using consumer key and secrete key
auth = tweepy.OAuthHandler(consumer_key, secret_key, callback_uri)
#grabs a url after successful authentication
redirect_url = auth.get_authorization_url()
webbrowser.open_new_tab(redirect_url)
print(redirect_url)
#click on url and grab pin number
#pin number will change every loggin 
user_pin_input = input("Enter Pin Number: ")

#now the auth object has access to your consumer and secret token:
auth.get_access_token(user_pin_input)
print(auth.access_token)
print()
print(auth.access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)
#now you have access to everything in the api. 
#see documentation

https://api.twitter.com/oauth/authorize?oauth_token=Tn189AAAAAABI4eZAAABdezyBTU
Enter Pin Number: 2589782
1317128139261304832-FLljxAQviuMVHQoiRpeFghc6VoDODQ

h7bZ20TVfLKTW19aDfnNcX3JAyHc2auYDgG7ByRJfKF5l


In [ ]:
#!pip install m3inference

In [ ]:
#!git clone https://github.com/zijwang/m3inference.git

In [ ]:
#script for resizing profile pictures: 
#!python scripts/preprocess.py --source_dir test/pic/ --output_dir test/pic_resized/ --jsonl_path test/data.jsonl --jsonl_outpath test/data_resized.jsonl --verbose

In [ ]:
#test 1 
#from the same root as /test/ 

from m3inference import M3Inference
import pprint
m3 = M3Inference() # see docstring for details
pred = m3.infer('./test/data_resized.jsonl') # also see docstring for details
pprint.pprint(pred)

In [ ]:
#test 2 
from m3inference import M3Twitter
import pprint

m3twitter=M3Twitter(cache_dir="profile_images") #Change the cache_dir parameter to control where profile images are downloaded
m3twitter.transform_jsonl(input_file="nuclear_test/example_tweets.jsonl",output_file="nuclear_test/m3_input.jsonl")

predictions = m3twitter.infer("nuclear_test/m3_input.jsonl")
pprint.pprint(predictions) #Same method as M3Inference.infer(...)

### 1. Get json dump of users from CSV of tweets 

In [3]:
import pandas as pd 
import json 

start = time.perf_counter()

filename = "C:/Users/sbgas/OneDrive/Desktop/total_11-21-20.csv"
df = pd.read_csv(filename, encoding='utf-8', index_col =0, engine='python')
df.drop_duplicates(subset=['screen_name'], inplace=True) #get only unique screennames

data = []
outputfile = "C:/Users/sbgas/OneDrive/Desktop/m3inference-master/nuclear/users.jsonl"
with open(outputfile, 'w') as f:
    for index, row in df.iterrows():
        #try:
        user = api.get_user(row['screen_name'])
        data.append(json.dumps(user._json)+'\n') #makes this a jsonl file 
        #except:
        #    print("Could not find ", row['screen_name'])

    f.writelines(data)
    f.close()
print(time.perf_counter()-start, "seconds")

TweepError: [{'code': 50, 'message': 'User not found.'}]

### 2. use M3 to get profile pictures

In [5]:
#make predictions
from m3inference import M3Twitter
import pprint

start = time.perf_counter()

inputjsonl = outputfile #from previous cell
m3_model_file = "C:/Users/sbgas/OneDrive/Desktop/m3inference-master/nuclear/m3_input.jsonl"

m3twitter=M3Twitter(cache_dir="C:/Users/sbgas/OneDrive/Desktop/m3inference-master/nuclear/pic") #Change the cache_dir parameter to control where profile images are downloaded
m3twitter.transform_jsonl(input_file=inputjsonl, output_file=m3_model_file) 

print(time.perf_counter()-start, "seconds")

11/21/2020 22:26:07 - INFO - m3inference.m3inference -   Version 1.0
11/21/2020 22:26:07 - INFO - m3inference.m3inference -   Running on cpu.
11/21/2020 22:26:07 - INFO - m3inference.m3inference -   Will use full M3 model.
11/21/2020 22:26:08 - INFO - m3inference.m3inference -   Model full_model exists at C:\Users\sbgas/m3/models/full_model.mdl.
11/21/2020 22:26:08 - INFO - m3inference.utils -   Checking MD5 for model full_model at C:\Users\sbgas/m3/models/full_model.mdl
11/21/2020 22:26:08 - INFO - m3inference.utils -   MD5s match.
11/21/2020 22:26:08 - INFO - m3inference.m3inference -   Loaded pretrained weight at C:\Users\sbgas/m3/models/full_model.mdl


1.3715375999999964 seconds


### 2.5. Clean jsonl file. Delete missing image paths. Remove emojis from desc and name. Remove all .GIF file types. 

Some users have protected accounts or are too obscure for Twitter's search People bar, so we have to remove them and hand label the demos of those users 

In [40]:
def deEmojify(text):
    import re
    regrex_pattern = re.compile(pattern = "["
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251" 
                           "]+", flags = re.UNICODE)

    return regrex_pattern.sub(r'',text)

In [72]:
#find which profile pictures are missing and delete manually 
#m3_model_file is the jsonl file that we just created with M3 
import jsonlines
import os 

start = time.perf_counter()
paths = [] 

with jsonlines.open("C:/Users/sbgas/OneDrive/Desktop/m3inference-master/nuclear/m3_input.jsonl") as reader:
    for obj in reader:
        if not os.path.isfile(obj['img_path']): #print all paths to non-existent files 
            paths.append(obj['img_path'])
            
print(len(paths), "inputs in the m3_inputs.jsonl file  that aren't in pic/" )
print(time.perf_counter()-start, "seconds")

0 inputs in the m3_inputs.jsonl file  that aren't in pic/
0.3522895000005519 seconds


In [68]:
def remove_records(input_file, output_file, paths):
    #input file is the m3_input.jsonl file
    #paths is the list of paths to photos that are missing in the pic directory 
    
    #copy over only the lines that dont contain the missing image 
    import jsonlines, json

    with jsonlines.open(input_file) as reader:
        with open(output_file, mode='w', encoding='utf-8') as writer:
            for obj in reader:
                if str(obj['img_path']) not in paths:
                    #remove emoji and write obj to new file 
                    obj['description'] = deEmojify(obj['description'])
                    obj['name'] = deEmojify(obj['name'])
                    writer.write(json.dumps(obj) + "\n")
            writer.close()
        reader.close()
    print("Done. Check to make sure there are no more incorrect paths")

In [69]:
remove_records("C:/Users/sbgas/OneDrive/Desktop/m3inference-master/nuclear/m3_input - Copy.jsonl", "C:/Users/sbgas/OneDrive/Desktop/m3inference-master/nuclear/m3_input.jsonl", paths)

Done. Check to make sure there are no more incorrect paths


### Remove .GIF files 

In [ ]:
### TBD 
#loop through m3_input.jsonl and check "img_url" to make sure it doesn't end in ".gif"

### 3. use M3 to make predictions 

In [1]:
#optional:
from m3inference import M3Twitter
import pprint, time

m3_model_file = "C:/Users/sbgas/OneDrive/Desktop/m3inference-master/nuclear/m3_input.jsonl"
m3twitter=M3Twitter(cache_dir="C:/Users/sbgas/OneDrive/Desktop/m3inference-master/nuclear/pic") #Change the cache_dir parameter to control where profile images are downloaded

11/23/2020 10:07:56 - INFO - m3inference.m3inference -   Version 1.0
11/23/2020 10:07:56 - INFO - m3inference.m3inference -   Running on cpu.
11/23/2020 10:07:56 - INFO - m3inference.m3inference -   Will use full M3 model.
11/23/2020 10:07:57 - INFO - m3inference.m3inference -   Model full_model exists at C:\Users\sbgas/m3/models/full_model.mdl.
11/23/2020 10:07:57 - INFO - m3inference.utils -   Checking MD5 for model full_model at C:\Users\sbgas/m3/models/full_model.mdl
11/23/2020 10:07:57 - INFO - m3inference.utils -   MD5s match.
11/23/2020 10:07:58 - INFO - m3inference.m3inference -   Loaded pretrained weight at C:\Users\sbgas/m3/models/full_model.mdl


In [2]:
start = time.perf_counter()
predictions = m3twitter.infer(m3_model_file)

print(time.perf_counter()-start, "seconds")

11/23/2020 10:08:05 - INFO - m3inference.dataset -   5335 data entries loaded.
Predicting...: 100%|█████████████████████████████████████████████████████████████████| 334/334 [35:05<00:00,  6.31s/it]


2106.234326 seconds


In [10]:
pprint.pprint(predictions)

OrderedDict([('1170764586',
              {'age': {'19-29': 0.6447,
                       '30-39': 0.0947,
                       '<=18': 0.2297,
                       '>=40': 0.0309},
               'gender': {'female': 0.0, 'male': 1.0},
               'org': {'is-org': 0.0002, 'non-org': 0.9998}}),
             ('2172265702',
              {'age': {'19-29': 0.1805,
                       '30-39': 0.4693,
                       '<=18': 0.0105,
                       '>=40': 0.3396},
               'gender': {'female': 0.9862, 'male': 0.0138},
               'org': {'is-org': 0.0, 'non-org': 1.0}}),
             ('59800550',
              {'age': {'19-29': 0.0833,
                       '30-39': 0.4187,
                       '<=18': 0.0033,
                       '>=40': 0.4947},
               'gender': {'female': 0.0018, 'male': 0.9982},
               'org': {'is-org': 0.0057, 'non-org': 0.9943}}),
             ('717242270',
              {'age': {'19-29': 0.1686,
              

                       '30-39': 0.0017,
                       '<=18': 0.0011,
                       '>=40': 0.993},
               'gender': {'female': 0.5043, 'male': 0.4957},
               'org': {'is-org': 0.8973, 'non-org': 0.1027}}),
             ('3342682019',
              {'age': {'19-29': 0.0156,
                       '30-39': 0.7389,
                       '<=18': 0.0042,
                       '>=40': 0.2414},
               'gender': {'female': 0.0004, 'male': 0.9996},
               'org': {'is-org': 0.0008, 'non-org': 0.9992}}),
             ('3379924216',
              {'age': {'19-29': 0.3028,
                       '30-39': 0.3309,
                       '<=18': 0.1563,
                       '>=40': 0.21},
               'gender': {'female': 0.5152, 'male': 0.4848},
               'org': {'is-org': 0.3618, 'non-org': 0.6382}}),
             ('1199747007058980864',
              {'age': {'19-29': 0.143,
                       '30-39': 0.6916,
                      

                       '<=18': 0.0015,
                       '>=40': 0.9836},
               'gender': {'female': 0.0047, 'male': 0.9953},
               'org': {'is-org': 0.163, 'non-org': 0.837}}),
             ('888077810195206153',
              {'age': {'19-29': 0.0444,
                       '30-39': 0.0634,
                       '<=18': 0.0101,
                       '>=40': 0.8821},
               'gender': {'female': 0.0323, 'male': 0.9677},
               'org': {'is-org': 0.8265, 'non-org': 0.1735}}),
             ('190317730',
              {'age': {'19-29': 0.0205,
                       '30-39': 0.0064,
                       '<=18': 0.0159,
                       '>=40': 0.9572},
               'gender': {'female': 0.1363, 'male': 0.8637},
               'org': {'is-org': 0.9536, 'non-org': 0.0464}}),
             ('85702248',
              {'age': {'19-29': 0.0002,
                       '30-39': 0.0005,
                       '<=18': 0.0,
                       '>=40

               'org': {'is-org': 0.0003, 'non-org': 0.9997}}),
             ('1021137194138898440',
              {'age': {'19-29': 0.0918,
                       '30-39': 0.0108,
                       '<=18': 0.8621,
                       '>=40': 0.0353},
               'gender': {'female': 0.0795, 'male': 0.9205},
               'org': {'is-org': 0.0032, 'non-org': 0.9968}}),
             ('19379462',
              {'age': {'19-29': 0.6971,
                       '30-39': 0.1596,
                       '<=18': 0.0819,
                       '>=40': 0.0615},
               'gender': {'female': 0.0323, 'male': 0.9677},
               'org': {'is-org': 0.0, 'non-org': 1.0}}),
             ('3309139560',
              {'age': {'19-29': 0.3433,
                       '30-39': 0.5192,
                       '<=18': 0.0258,
                       '>=40': 0.1117},
               'gender': {'female': 0.0002, 'male': 0.9998},
               'org': {'is-org': 0.0001, 'non-org': 0.9999}}),
   

             ('165934859',
              {'age': {'19-29': 0.6701,
                       '30-39': 0.1868,
                       '<=18': 0.0557,
                       '>=40': 0.0873},
               'gender': {'female': 0.8276, 'male': 0.1724},
               'org': {'is-org': 0.0036, 'non-org': 0.9964}}),
             ('901240694',
              {'age': {'19-29': 0.0,
                       '30-39': 0.0001,
                       '<=18': 0.0,
                       '>=40': 0.9999},
               'gender': {'female': 0.9985, 'male': 0.0015},
               'org': {'is-org': 0.001, 'non-org': 0.999}}),
             ('2587000682',
              {'age': {'19-29': 0.0243,
                       '30-39': 0.0361,
                       '<=18': 0.0035,
                       '>=40': 0.9361},
               'gender': {'female': 0.1656, 'male': 0.8344},
               'org': {'is-org': 0.0004, 'non-org': 0.9996}}),
             ('1305662590777987072',
              {'age': {'19-29': 0.0001,


...

                       '<=18': 0.2707,
                       '>=40': 0.0808},
               'gender': {'female': 0.0019, 'male': 0.9981},
               'org': {'is-org': 0.0002, 'non-org': 0.9998}}),
             ('45261535',
              {'age': {'19-29': 0.0637,
                       '30-39': 0.0653,
                       '<=18': 0.0893,
                       '>=40': 0.7817},
               'gender': {'female': 0.0836, 'male': 0.9164},
               'org': {'is-org': 0.8558, 'non-org': 0.1442}}),
             ('1052279755259736064',
              {'age': {'19-29': 0.0248,
                       '30-39': 0.3588,
                       '<=18': 0.0037,
                       '>=40': 0.6128},
               'gender': {'female': 0.4377, 'male': 0.5623},
               'org': {'is-org': 0.8789, 'non-org': 0.1211}}),
             ('20419776',
              {'age': {'19-29': 0.0114,
                       '30-39': 0.0105,
                       '<=18': 0.0011,
                       

             ('2724651985',
              {'age': {'19-29': 0.2687,
                       '30-39': 0.0885,
                       '<=18': 0.0025,
                       '>=40': 0.6403},
               'gender': {'female': 0.9972, 'male': 0.0028},
               'org': {'is-org': 0.0, 'non-org': 1.0}}),
             ('1211507457710678016',
              {'age': {'19-29': 0.0773,
                       '30-39': 0.0128,
                       '<=18': 0.1097,
                       '>=40': 0.8001},
               'gender': {'female': 0.0037, 'male': 0.9963},
               'org': {'is-org': 0.0061, 'non-org': 0.9939}}),
             ('1414650259',
              {'age': {'19-29': 0.1433,
                       '30-39': 0.6611,
                       '<=18': 0.1875,
                       '>=40': 0.0082},
               'gender': {'female': 0.002, 'male': 0.998},
               'org': {'is-org': 0.0003, 'non-org': 0.9997}}),
             ('1132661943624318976',
              {'age': {'19-29

                       '>=40': 0.9002},
               'gender': {'female': 0.0789, 'male': 0.9211},
               'org': {'is-org': 0.9615, 'non-org': 0.0385}}),
             ('22984215',
              {'age': {'19-29': 0.1059,
                       '30-39': 0.0678,
                       '<=18': 0.0309,
                       '>=40': 0.7954},
               'gender': {'female': 0.0291, 'male': 0.9709},
               'org': {'is-org': 0.9076, 'non-org': 0.0924}}),
             ('17054817',
              {'age': {'19-29': 0.0064,
                       '30-39': 0.0011,
                       '<=18': 0.0079,
                       '>=40': 0.9846},
               'gender': {'female': 0.0561, 'male': 0.9439},
               'org': {'is-org': 0.9375, 'non-org': 0.0625}}),
             ('86364947',
              {'age': {'19-29': 0.0003,
                       '30-39': 0.0022,
                       '<=18': 0.0,
                       '>=40': 0.9974},
               'gender': {'female': 

                       '30-39': 0.1085,
                       '<=18': 0.0339,
                       '>=40': 0.7745},
               'gender': {'female': 0.0183, 'male': 0.9817},
               'org': {'is-org': 0.8706, 'non-org': 0.1294}}),
             ('335601509',
              {'age': {'19-29': 0.0328,
                       '30-39': 0.9082,
                       '<=18': 0.0228,
                       '>=40': 0.0362},
               'gender': {'female': 0.0174, 'male': 0.9826},
               'org': {'is-org': 0.0002, 'non-org': 0.9998}}),
             ('15807344',
              {'age': {'19-29': 0.0316,
                       '30-39': 0.004,
                       '<=18': 0.0347,
                       '>=40': 0.9297},
               'gender': {'female': 0.7707, 'male': 0.2293},
               'org': {'is-org': 0.9794, 'non-org': 0.0206}}),
             ('1267650610859556870',
              {'age': {'19-29': 0.1054,
                       '30-39': 0.0041,
                      

                       '30-39': 0.0475,
                       '<=18': 0.0212,
                       '>=40': 0.834},
               'gender': {'female': 0.0492, 'male': 0.9508},
               'org': {'is-org': 0.8799, 'non-org': 0.1201}}),
             ('12336532',
              {'age': {'19-29': 0.6111,
                       '30-39': 0.1265,
                       '<=18': 0.1644,
                       '>=40': 0.0981},
               'gender': {'female': 0.0402, 'male': 0.9598},
               'org': {'is-org': 0.1071, 'non-org': 0.8929}}),
             ('2898746618',
              {'age': {'19-29': 0.0101,
                       '30-39': 0.024,
                       '<=18': 0.0062,
                       '>=40': 0.9597},
               'gender': {'female': 0.0178, 'male': 0.9822},
               'org': {'is-org': 0.0032, 'non-org': 0.9968}}),
             ('564269780',
              {'age': {'19-29': 0.0077,
                       '30-39': 0.0031,
                       '<=18': 0

In [3]:
len(predictions)

5335

In [66]:
#example of output
predictions['1278194570396864513']

{'gender': {'male': 0.0264, 'female': 0.9736},
 'age': {'<=18': 0.0011, '19-29': 0.0195, '30-39': 0.9698, '>=40': 0.0096},
 'org': {'non-org': 0.9997, 'is-org': 0.0003}}

In [6]:
#save prediction dictionary
import json
with open("user_demo_dict.json", mode='w', encoding='utf-8') as writer:
        json.dump(predictions,writer)

In [7]:
#check to make sure it saved properly
with open("user_demo_dict.json", mode='r', encoding='utf-8') as reader:
        pred = json.load(reader)
        print(len(pred), type(pred))

5335 <class 'dict'>


### 4. add predictions to df of tweets 

In [249]:
#"filename" set in the cell above
import pandas as pd
filename = "C:/Users/sbgas/OneDrive/Desktop/Total_11-23-20.csv"
df = pd.read_csv(filename, encoding='utf-8', dtype={'user_id': str}, index_col =0, engine='python')

In [250]:
df.head(2)

,time_retrieved,text,created_at,favorite_count,retweet_count,place,coordinates,user_id,screen_name,user_description,user_follower_count,verified,user_location,City,State,Country
status_id,,,,,,,,,,,,,,,,
1327298633683791872,2020-11-16 21:14,Facts: \n\nChernobyl doesn't prove that nuclea...,2020-11-13 17:13:48,348.0,74.0,NaN,NaN,41160276,prageru,Free Videos. Free Minds.,563593.0,True,Los Angeles,Los Angeles,CA,USA
1328517972026159104,2020-11-16 21:14,Microreactors are the smallest class of nuclea...,2020-11-17 01:59:01,20.0,6.0,NaN,NaN,1100064156504543233,GovNuclear,Making nuclear cool again 😎. Advancing new tec...,12137.0,True,"Washington, DC",DC,MD,USA


In [251]:
import numpy as np 

gender_list = []
age_list = []
org_list = [] 
count = 0 
for index, row in df.iterrows():
    try: 
        demos = predictions[str(int(row['user_id']))]
        count+=1
        if demos['org']['is-org'] > 0.50: #<--- play with this to see if it changes accuracy significantly 
            org = True
            gender = np.NaN
            age = np.NaN
        else:
            org = False
            gender = max(demos['gender'], key=demos['gender'].get) #get key with the maximum value 
            age = max(demos['age'], key=demos['age'].get) #get key with the maximum value 
    except:
        gender = np.NaN
        age = np.NaN
        org = np.NaN
        
    gender_list.append(gender)
    age_list.append(age)
    org_list.append(org)
    
print(count, len(predictions))
print(df.shape, len(gender_list),len(age_list),len(org_list) )


18818 5335
(19285, 16) 19285 19285 19285


In [252]:
df['gender'] = gender_list
df['age'] = age_list
df['org'] = org_list
df.head()

,time_retrieved,text,created_at,favorite_count,retweet_count,place,coordinates,user_id,screen_name,user_description,user_follower_count,verified,user_location,City,State,Country,gender,age,org
status_id,,,,,,,,,,,,,,,,,,,
1327298633683791872,2020-11-16 21:14,Facts: \n\nChernobyl doesn't prove that nuclea...,2020-11-13 17:13:48,348.0,74.0,NaN,NaN,41160276,prageru,Free Videos. Free Minds.,563593.0,True,Los Angeles,Los Angeles,CA,USA,NaN,NaN,True
1328517972026159104,2020-11-16 21:14,Microreactors are the smallest class of nuclea...,2020-11-17 01:59:01,20.0,6.0,NaN,NaN,1100064156504543233,GovNuclear,Making nuclear cool again 😎. Advancing new tec...,12137.0,True,"Washington, DC",DC,MD,USA,NaN,NaN,True
1328409437757968385,2020-11-16 21:14,Nice resource on global nuclear energy statist...,2020-11-16 18:47:45,3.0,1.0,NaN,NaN,1100064156504543233,GovNuclear,Making nuclear cool again 😎. Advancing new tec...,12137.0,True,"Washington, DC",DC,MD,USA,NaN,NaN,True
1328407132925026311,2020-11-16 21:14,Excited to see @ENERGY and @NASA's strong part...,2020-11-16 18:38:35,0.0,15.0,NaN,NaN,1100064156504543233,GovNuclear,Making nuclear cool again 😎. Advancing new tec...,12137.0,True,"Washington, DC",DC,MD,USA,NaN,NaN,True
1328367476908896260,2020-11-16 21:14,VIRTUAL EVENT: @ENERGY and industry experts di...,2020-11-16 16:01:01,11.0,4.0,NaN,NaN,1100064156504543233,GovNuclear,Making nuclear cool again 😎. Advancing new tec...,12137.0,True,"Washington, DC",DC,MD,USA,NaN,NaN,True


In [253]:
df['age'].value_counts()

>=40     6682
30-39    2578
19-29    1980
<=18      818
Name: age, dtype: int64

In [254]:
df['gender'].value_counts()

male      8689
female    3369
Name: gender, dtype: int64

In [255]:
#80%
df.org.value_counts()

False    12058
True      6760
Name: org, dtype: int64

In [256]:
#60%
df.org.value_counts()

False    12058
True      6760
Name: org, dtype: int64

In [257]:
#at least 3 wrong out of 66 predictions. 3 of these 9 "False" were actually organizations.
#... and couldn't get profile pic for 2 our of 68 = 3%
df.org.value_counts()

False    12058
True      6760
Name: org, dtype: int64

### save to csv 

In [258]:
df.to_csv("demos.csv") #filename is the same file you read from 

In [260]:
df_users = df.drop_duplicates(subset=['screen_name'])
df_users.shape

(5518, 19)

In [261]:
df_users = df_users[['user_id', 'screen_name', 'user_description', 'user_follower_count', 'verified', 'City', 'State', 'Country', 'gender', 'age', 'org']]
df_users.shape

(5518, 11)

In [262]:
df_users.set_index('screen_name', inplace=True)
df_users.head()

,user_id,user_description,user_follower_count,verified,City,State,Country,gender,age,org
screen_name,,,,,,,,,,
prageru,41160276,Free Videos. Free Minds.,563593.0,True,Los Angeles,CA,USA,NaN,NaN,True
GovNuclear,1100064156504543233,Making nuclear cool again 😎. Advancing new tec...,12137.0,True,DC,MD,USA,NaN,NaN,True
NEI,27688281,⁣Our vision is simple: A world powered by clea...,47014.0,True,DC,MD,USA,NaN,NaN,True
AaronMehta,103016675,Deputy editor and Pentagon Correspondent @Defe...,28842.0,True,DC,MD,USA,male,19-29,False
WashTimes,14662354,"Reliable reporting, hard-hitting analysis & br...",415154.0,True,DC,MD,USA,NaN,NaN,True


In [263]:
df_users.to_csv("user_demos50.csv")

In [2]:
import pandas as pd
df_users = pd.read_csv("user_demos50.csv", dtype={"user_id":str})

In [13]:
df_missing_labels = df_users[df_users['org'] != True]
df_missing_labels = df_missing_labels[df_missing_labels['org'] != False]

In [21]:
df_missing_labels.shape

(184, 11)

In [8]:
df_missing_labels.to_csv("missing_demos.csv")

### get user breakdowns

In [25]:
num_users = len(df_users['org']) - len(df_missing_labels['org'])
print(num_users)

5334


In [6]:
df_users['org'].value_counts()

False    4566
True      768
Name: org, dtype: int64

In [32]:
print("Pct Org:", len(df_users[df_users['org']==True]) / num_users * 100)
num_non_org = len(df_users[df_users['org']==False]) 
print("Pct Non-Org:", num_non_org / num_users * 100)

Pct Org: 14.39820022497188
Pct Non-Org: 85.60179977502813


In [40]:
print("Pct male:", len(df_users[df_users['gender']=="male"]) / num_non_org * 100)
print("Pct female:", len(df_users[df_users['gender']=="female"]) / num_non_org * 100)

Pct male: 67.34559789750328
Pct female: 32.65440210249672


In [41]:
print("Pct <=18:", len(df_users[df_users['age']=="<=18"]) / num_non_org * 100)
print("Pct 19-29:", len(df_users[df_users['age']=="19-29"]) / num_non_org * 100)
print("Pct 30-39:", len(df_users[df_users['age']=="30-39"]) / num_non_org * 100)
print("Pct >=40:", len(df_users[df_users['age']==">=40"]) / num_non_org * 100)

Pct <=18: 9.154621112571178
Pct 19-29: 20.039421813403415
Pct 30-39: 20.105124835742444
Pct >=40: 50.700832238282956
